## Introduction

In this notebook, my goal is to analyze a historical dataset covering both the Winter and Summer Olympics from 1896 to 2016. I will primarily be using SQLite queries to extract key insights from the data. The dataset was scraped from www.sports-reference.com, and I found it on Kaggle [here]( https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results?select=noc_regions.csv)

**About the dataset:** This dataset contains 2,711,116 rows and 15 columns. Each row represents an individual athlete's performance at a single event at a single Olympic games. The 15 
columns represent the following: <br>
1. **ID** - unique numerical code for each athlete
2. **Name** - athlete's name
3. **Sex** - M or F
4. **Age** - athlete's age at time of olympics
5. **Height** - in cm
6. **Weight** - in kg
7. **Team** - athlete's team/country name
8. **NOC** - National Olympic Committee 3-letter coe
9. **Games** - year and winter or summer
10. **Year**
11. **Season** - winter or summer
12. **City** - host city
13. **Sport**
14. **Event**
15. **Medal** - NA, Bronze, Silver, or Gold

In addition to the main dataset, there is also another file called "noc_regions.csv" that maps out all NOCs to their respective countries


## Load In Data

##### Import Statements

In [1]:
import pandas 
import sqlite3

##### Read in the data

In [2]:
## Read the files

olympians = pandas.read_csv('athlete_events.csv',index_col=0)
countries = pandas.read_csv('noc_regions.csv',index_col=0)

##### Create SQL Enginne

In [3]:
# import sqlalchemy and create a sqlite engine
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# export the dataframe to the sqlite engine
olympians.to_sql("olympians", con =engine)
countries.to_sql("countries", con =engine)

In [4]:
## Run query function
def run_query(a):
    df_sql = pandas.read_sql_query(a,con=engine,index_col = None)
    return df_sql

## Data Cleaning

##### Check dimensions

In [5]:
olympians.shape

(271116, 14)

In [6]:
countries.shape

(230, 2)

##### Check first few values

In [7]:
olympians.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
ID,,,,,,,,,,,,,,
1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [8]:
countries.head()

,region,notes
NOC,,
AFG,Afghanistan,NaN
AHO,Curacao,Netherlands Antilles
ALB,Albania,NaN
ALG,Algeria,NaN
AND,Andorra,NaN


##### Check for null values

In [9]:
olympians.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271116 entries, 1 to 135571
Data columns (total 14 columns):
Name      271116 non-null object
Sex       271116 non-null object
Age       261642 non-null float64
Height    210945 non-null float64
Weight    208241 non-null float64
Team      271116 non-null object
NOC       271116 non-null object
Games     271116 non-null object
Year      271116 non-null int64
Season    271116 non-null object
City      271116 non-null object
Sport     271116 non-null object
Event     271116 non-null object
Medal     39783 non-null object
dtypes: float64(3), int64(1), object(10)
memory usage: 31.0+ MB


In [10]:
age_null = """

SELECT COUNT(DISTINCT (ID)) AS ID
    ,(100 * COUNT(DISTINCT (ID))) / (
        SELECT COUNT(DISTINCT (ID))
        FROM olympians
        ) AS Percentage
FROM olympians
WHERE Age IS NULL

"""
run_query(age_null)

,ID,Percentage
0,6368,4


In [11]:
height_null = """

SELECT COUNT(DISTINCT (ID)) AS ID
    ,(100 * COUNT(DISTINCT (ID))) / (
        SELECT COUNT(DISTINCT (ID))
        FROM olympians
        ) AS Percentage
FROM olympians
WHERE Height IS NULL

"""
run_query(height_null)

,ID,Percentage
0,33916,25


In [12]:
weight_null = """

SELECT COUNT(DISTINCT (ID)) AS ID
    ,(100 * COUNT(DISTINCT (ID))) / (
        SELECT COUNT(DISTINCT (ID))
        FROM olympians
        ) AS Percentage
FROM olympians
WHERE Weight IS NULL

"""
run_query(weight_null)

,ID,Percentage
0,34885,25


In [13]:
medal_null = """

SELECT COUNT(DISTINCT (ID)) AS ID
    ,(100 * COUNT(DISTINCT (ID))) / (
        SELECT COUNT(DISTINCT (ID))
        FROM olympians
        ) AS Percentage
FROM olympians
WHERE Medal IS NULL

"""
run_query(medal_null)

,ID,Percentage
0,121097,89


From the above, we can see that "Age", "Height", "Weight", and "Medal" have null values. Regarding "Medal", we can assume that null values mean that the athlete did not win a medal, so that is not a problem. For the others, I will impute the missing values with the median of the non-null values of that column. Although I will be using this approach, this decreases my confidence in the accuracy of those columns as a whole, so I will not be computing their averages in the below exercises for fear of misrepresenting the actual figures

##### Imputing Values for "Age", "Height", and "Weight"

In [14]:
olympians['Age'].fillna(olympians['Age'].median(),inplace= True)
olympians['Height'].fillna(olympians['Height'].median(),inplace= True)
olympians['Weight'].fillna(olympians['Weight'].median(),inplace= True)
olympians.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271116 entries, 1 to 135571
Data columns (total 14 columns):
Name      271116 non-null object
Sex       271116 non-null object
Age       271116 non-null float64
Height    271116 non-null float64
Weight    271116 non-null float64
Team      271116 non-null object
NOC       271116 non-null object
Games     271116 non-null object
Year      271116 non-null int64
Season    271116 non-null object
City      271116 non-null object
Sport     271116 non-null object
Event     271116 non-null object
Medal     39783 non-null object
dtypes: float64(3), int64(1), object(10)
memory usage: 31.0+ MB


## Data Exploration with SQL

##### How many Summer, Winter, and Total Olympic games does our data include?

In [15]:
list_of_olympics = """

SELECT (
        SELECT COUNT(DISTINCT Games)
        FROM olympians
        WHERE Season = 'Summer'
        ) AS Summer_Olympics
    ,(
        SELECT COUNT(DISTINCT Games)
        FROM olympians
        WHERE Season = 'Winter'
        ) AS Winter_Olympics
    ,(
        SELECT COUNT(DISTINCT Games)
        FROM olympians
        ) AS Total_Olympics;
      
"""
run_query(list_of_olympics)

,Summer_Olympics,Winter_Olympics,Total_Olympics
0,29,22,51


##### How many teams, sports, and events have there been?

In [16]:
num_teams = """

SELECT COUNT(DISTINCT noc) AS Num_of_Teams
FROM countries

"""
run_query(num_teams)

,Num_of_Teams
0,230


In [17]:
num_of_sports = """

SELECT COUNT(DISTINCT Sport) AS Num_of_Sports
FROM olympians

"""
run_query(num_of_sports)

,Num_of_Sports
0,66


In [18]:
num_of_events = """

SELECT COUNT(DISTINCT Event) AS Num_of_Events
FROM olympians

"""
run_query(num_of_events)

,Num_of_Events
0,765


##### Which cities have hosted the olympics more than once?

In [19]:
host_rankings = """

WITH host_rankings
AS (
    SELECT (City)
        ,(Games)
    FROM olympians
    GROUP BY City
        ,Games
    ORDER BY Games
    )
SELECT City
    ,COUNT(City) AS num_hosted
FROM host_rankings
GROUP BY City
ORDER BY num_hosted DESC LIMIT 8;

"""
run_query(host_rankings)

,City,num_hosted
0,London,3
1,Athina,3
2,Stockholm,2
3,Sankt Moritz,2
4,Paris,2
5,Los Angeles,2
6,Lake Placid,2
7,Innsbruck,2


##### How many countries have participated in the Olympics over the years?

In [20]:
country_participation = '''

SELECT DISTINCT (Games)
    ,COUNT(DISTINCT (noc)) AS Number_of_Countries
FROM olympians
GROUP BY Games
ORDER BY Games;

'''
run_query(country_participation)

,Games,Number_of_Countries
0,1896 Summer,12
1,1900 Summer,31
2,1904 Summer,15
3,1906 Summer,21
4,1908 Summer,22
5,1912 Summer,29
6,1920 Summer,29
7,1924 Summer,45
8,1924 Winter,19
9,1928 Summer,46


##### Which countries have participated in the most Olympics? Have any countries participated in all of them?

In [21]:
most_participated = '''

SELECT DISTINCT (NOC)
    ,COUNT(DISTINCT (Games)) AS participation_number
FROM olympians
GROUP BY NOC
ORDER BY participation_number DESC LIMIT 10;

'''
run_query(most_participated)

,NOC,participation_number
0,SUI,51
1,ITA,51
2,GBR,51
3,FRA,51
4,USA,50
5,SWE,50
6,AUT,50
7,NOR,49
8,HUN,49
9,CAN,49


Switzerland, Italy, Great Britain, and France have participated in all 51 Olympic Games in our dataset

##### Which country has had the most athletes participate?

In [22]:
athletes_by_country = """

SELECT data.region
    ,COUNT(DISTINCT (ID)) AS Num_of_Athletes
FROM olympians
INNER JOIN countries data ON olympians.NOC = data.NOC
GROUP BY data.region
ORDER BY Num_of_Athletes DESC LIMIT 10;

"""
run_query(athletes_by_country)

,region,Num_of_Athletes
0,USA,9653
1,Germany,7575
2,UK,6281
3,France,6170
4,Russia,5610
5,Italy,4935
6,Canada,4812
7,Japan,4067
8,Australia,3870
9,Sweden,3787


##### Who are the Oldest Olympians in our data?

In [23]:
oldest_olympians = """

SELECT DISTINCT NAME
    ,CAST(age AS INT) AS Age
    ,Sex
    ,event
    ,noc AS Country
    ,games
    ,medal
FROM olympians
WHERE age <> 'NA'
ORDER BY age DESC LIMIT 10;

"""
run_query(oldest_olympians)

,Name,Age,Sex,Event,Country,Games,Medal
0,John Quincy Adams Ward,97,M,"Art Competitions Mixed Sculpturing, Statues",USA,1928 Summer,None
1,Winslow Homer,96,M,"Art Competitions Mixed Painting, Unknown Event",USA,1932 Summer,None
2,Thomas Cowperthwait Eakins,88,M,"Art Competitions Mixed Painting, Unknown Event",USA,1932 Summer,None
3,George Denholm Armour,84,M,"Art Competitions Mixed Painting, Unknown Event",GBR,1948 Summer,None
4,Robert Tait McKenzie,81,M,"Art Competitions Mixed Sculpturing, Unknown Event",CAN,1948 Summer,None
5,Louis Tauzin,81,M,Art Competitions Mixed Sculpturing,FRA,1924 Summer,None
6,Max Liebermann,80,M,"Art Competitions Mixed Painting, Graphic Arts",GER,1928 Summer,None
7,Charles Henry Niehaus,77,M,"Art Competitions Mixed Sculpturing, Unknown Event",USA,1932 Summer,None
8,Albin Dbrich,76,M,"Art Competitions Mixed Sculpturing, Unknown Event",AUT,1948 Summer,None
9,George Alfred Elcock,76,M,"Art Competitions Mixed Painting, Unknown Event",GBR,1932 Summer,None


##### Who are the Youngest Olympians in the data?

In [24]:
youngest_olympians = """

SELECT DISTINCT NAME
    ,CAST(age AS INT) AS Age
    ,Sex
    ,event
    ,noc AS Country
    ,games
    ,medal
FROM olympians
WHERE age <> 'NA'
ORDER BY age LIMIT 10;

"""
run_query(youngest_olympians)

,Name,Age,Sex,Event,Country,Games,Medal
0,Dimitrios Loundras,10,M,"Gymnastics Men's Parallel Bars, Teams",GRE,1896 Summer,Bronze
1,Magdalena Cecilia Colledge,11,F,Figure Skating Women's Singles,GBR,1932 Winter,None
2,Carlos Bienvenido Front Barrera,11,M,Rowing Men's Coxed Eights,ESP,1992 Summer,None
3,Luigina Giavotti,11,F,Gymnastics Women's Team All-Around,ITA,1928 Summer,Silver
4,"Sonja Henie (-Topping, -Gardiner, -Onstad)",11,F,Figure Skating Women's Singles,NOR,1924 Winter,None
5,Beatrice Hutiu,11,F,Figure Skating Women's Singles,ROU,1968 Winter,None
6,Etsuko Inada,11,F,Figure Skating Women's Singles,JPN,1936 Winter,None
7,Liu Luyang,11,F,Figure Skating Mixed Ice Dancing,CHN,1988 Winter,None
8,Marcelle Matthews,11,F,Figure Skating Mixed Pairs,RSA,1960 Winter,None
9,Megan Olwen Devenish Taylor (-Mandeville-Ellis),11,F,Figure Skating Women's Singles,GBR,1932 Winter,None


##### Who are the heaviest Olympians in the data?

In [25]:
heaviest_olympians = """

SELECT DISTINCT NAME
    ,Weight AS Weight_kg
    ,CAST(weight * 2.20 AS DECIMAL) AS Weight_lbs
    ,event
    ,noc AS Country
    ,games
    ,medal
FROM olympians
WHERE weight <> 'NA'
ORDER BY weight_kg DESC LIMIT 10;

"""
run_query(heaviest_olympians)

,Name,Weight_kg,Weight_lbs,Event,Country,Games,Medal
0,"Ricardo Blas, Jr.",214.0,470.8,Judo Men's Heavyweight,GUM,2008 Summer,None
1,"Ricardo Blas, Jr.",214.0,470.8,Judo Men's Heavyweight,GUM,2012 Summer,None
2,Aytami Ruano Vega,198.0,435.6,Judo Men's Heavyweight,ESP,2004 Summer,None
3,Marek Galiski,190.0,418.0,"Wrestling Men's Super-Heavyweight, Greco-Roman",POL,1980 Summer,None
4,"Christopher J. ""Chris"" Taylor",182.0,400.4,"Wrestling Men's Super-Heavyweight, Greco-Roman",USA,1972 Summer,None
5,"Christopher J. ""Chris"" Taylor",182.0,400.4,"Wrestling Men's Super-Heavyweight, Freestyle",USA,1972 Summer,Bronze
6,Valentyn Rusliakov,180.0,396.0,Judo Men's Heavyweight,UKR,2000 Summer,None
7,Leonel Wilfredo Ruz Carmona,178.0,391.6,Judo Men's Heavyweight,VEN,2004 Summer,None
8,"Marcus Jerrold ""Mark"" Henry",176.5,388.3,Weightlifting Men's Super-Heavyweight,USA,1992 Summer,None
9,"Marcus Jerrold ""Mark"" Henry",176.5,388.3,Weightlifting Men's Super-Heavyweight,USA,1996 Summer,None


##### Who are the lightest Olympians in the data?

In [26]:
lightest_olympians = """

SELECT DISTINCT NAME
    ,Weight AS Weight_kg
    ,CAST(weight * 2.20 AS DECIMAL) AS Weight_lbs
    ,event
    ,noc AS Country
    ,games
    ,medal
FROM olympians
WHERE weight <> 'NA'
ORDER BY weight_kg LIMIT 10;

"""
run_query(lightest_olympians)

,Name,Weight_kg,Weight_lbs,Event,Country,Games,Medal
0,Choi Myong-Hui,25.0,55.0,Gymnastics Women's Individual All-Around,PRK,1980 Summer,None
1,Choi Myong-Hui,25.0,55.0,Gymnastics Women's Team All-Around,PRK,1980 Summer,None
2,Choi Myong-Hui,25.0,55.0,Gymnastics Women's Floor Exercise,PRK,1980 Summer,None
3,Choi Myong-Hui,25.0,55.0,Gymnastics Women's Horse Vault,PRK,1980 Summer,None
4,Choi Myong-Hui,25.0,55.0,Gymnastics Women's Uneven Bars,PRK,1980 Summer,None
5,Choi Myong-Hui,25.0,55.0,Gymnastics Women's Balance Beam,PRK,1980 Summer,None
6,Anita Jokiel,28.0,61.6,Gymnastics Women's Individual All-Around,POL,1980 Summer,None
7,Anita Jokiel,28.0,61.6,Gymnastics Women's Team All-Around,POL,1980 Summer,None
8,Anita Jokiel,28.0,61.6,Gymnastics Women's Floor Exercise,POL,1980 Summer,None
9,Anita Jokiel,28.0,61.6,Gymnastics Women's Horse Vault,POL,1980 Summer,None


##### Who are the tallest Olympians in the data?

In [27]:
tallest_olympians = """

SELECT DISTINCT NAME
    ,height AS Height_cm
    ,CAST(height * 0.394 / 12 AS INT) AS Height_ft
    ,CAST((height * 0.394) % 12 AS INT) AS Height_in
    ,sport
    ,noc
    ,medal
FROM olympians
WHERE height <> 'NA'
ORDER BY height_ft DESC
    ,height_in DESC LIMIT 10

"""
run_query(tallest_olympians)

,Name,Height_cm,Height_ft,Height_in,Sport,NOC,Medal
0,Yao Ming,226.0,7,5,Basketball,CHN,None
1,Gunther Behnke,221.0,7,3,Basketball,GER,None
2,Tommy Loren Burleson,223.0,7,3,Basketball,USA,Silver
3,Roberto Dueas Hernndez,221.0,7,3,Basketball,ESP,None
4,Arvydas Romas Sabonis,223.0,7,3,Basketball,URS,Gold
5,Arvydas Romas Sabonis,223.0,7,3,Basketball,LTU,Bronze
6,Zhang Zhaoxu,221.0,7,3,Basketball,CHN,None
7,"Lucien James ""Luc"" Longley",220.0,7,2,Basketball,AUS,None
8,Dmitry Aleksandrovich Musersky,219.0,7,2,Volleyball,RUS,Gold
9,Viktor Aleksandrovich Pankrashkin,220.0,7,2,Basketball,URS,Gold


##### Who are the shortest Olympians in the data?

In [28]:
shortest_olympians = """

SELECT DISTINCT NAME
    ,height AS height_cm
    ,CAST(height * 0.394 / 12 AS INT) AS height_ft
    ,CAST((height * 0.394) % 12 AS INT) AS height_in
    ,sport
    ,noc
    ,medal
FROM olympians
WHERE height <> 'NA'
ORDER BY height_ft
    ,height_in DESC LIMIT 10

"""
run_query(shortest_olympians)

,Name,height_cm,height_ft,height_in,Sport,NOC,Medal
0,Moustafa Ali Abdel Halim,152.0,4,11,Weightlifting,EGY,None
1,Najim Abdulrazak Al-Enazy,150.0,4,11,Athletics,KUW,None
2,Sarah Abitbol,150.0,4,11,Figure Skating,FRA,None
3,"Folashade ""Shade"" Abugan",150.0,4,11,Athletics,NGR,None
4,Andreea Roxana Acatrinei,150.0,4,11,Gymnastics,ROU,None
5,Andreea Roxana Acatrinei,150.0,4,11,Gymnastics,ROU,Bronze
6,Selvyana Adrian-Sofyan,151.0,4,11,Shooting,INA,None
7,Esraa Ahmed El-Sayed,150.0,4,11,Weightlifting,EGY,None
8,Riaz Ahmed,150.0,4,11,Hockey,PAK,Gold
9,Riaz Ahmed,150.0,4,11,Hockey,PAK,Silver


##### Which countries have won the most medals?

In [29]:
medals_ranking = """

SELECT Noc as Country
    ,(SUM(Gold) + SUM(Silver) + SUM(Bronze)) AS 'Total'
    ,SUM(Gold) AS 'Gold'
    ,SUM(Silver) AS 'Silver'
    ,SUM(Bronze) AS 'Bronze'
FROM (
    SELECT Noc
        ,CASE 
            WHEN medals.Medal = 'Bronze'
                THEN medals.Total
            END AS 'Bronze'
        ,CASE 
            WHEN medals.Medal = 'Silver'
                THEN medals.Total
            END AS 'Silver'
        ,CASE 
            WHEN medals.Medal = 'Gold'
                THEN medals.Total
            END AS 'Gold'
    FROM (
        WITH medals AS (
                SELECT *
                    ,COUNT(*)
                FROM olympians
                GROUP BY Games
                    ,Noc
                    ,Medal
                    ,Event
                )
        SELECT *
            ,COUNT(Medal) AS Total
        FROM medals
        GROUP BY Games
            ,Noc
            ,Medal
            ,Event
        ORDER BY Noc
        ) AS medals
    ) AS data
GROUP BY Noc
ORDER BY Total DESC LIMIT 10;

"""
run_query(medals_ranking)

,Country,Total,Gold,Silver,Bronze
0,USA,2823,1131,901,791
1,URS,1197,471,373,353
2,GER,1004,319,345,340
3,GBR,919,289,320,310
4,FRA,879,264,286,329
5,ITA,722,256,225,241
6,SWE,657,200,215,242
7,CHN,595,239,184,172
8,RUS,584,202,183,199
9,GDR,519,192,165,162


##### Which athletes have won the most medals?

In [30]:
athletes_ranking = """

SELECT DISTINCT "ID"
    ,NAME
    ,count(Medal) AS Number_of_Medals
    ,region AS Country
FROM olympians AS athletes
LEFT JOIN countries ON athletes.NOC = countries.NOC
GROUP BY 1
    ,2
    ,4
ORDER BY 3 DESC LIMIT 10;

"""
run_query(athletes_ranking)

,ID,Name,Number_of_Medals,Country
0,94406,"Michael Fred Phelps, II",28,USA
1,67046,Larysa Semenivna Latynina (Diriy-),18,Russia
2,4198,Nikolay Yefimovich Andrianov,15,Russia
3,11951,Ole Einar Bjrndalen,13,Norway
4,74420,Edoardo Mangiarotti,13,Italy
5,89187,Takashi Ono,13,Japan
6,109161,Borys Anfiyanovych Shakhlin,13,Russia
7,23426,Natalie Anne Coughlin (-Hall),12,USA
8,35550,Birgit Fischer-Schmidt,12,Germany
9,57998,Sawao Kato,12,Japan


##### Medal breakdown by male vs. female

In [31]:
medals_by_sex = """

WITH if_else_cases
AS (
    SELECT ID
        ,NAME
        ,CASE 
            WHEN sex = 'F'
                THEN 'Female'
            ELSE 'Male'
            END AS Sex
        ,CASE 
            WHEN Medal = 'Gold'
                THEN '1'
            ELSE 0
            END AS Gold
        ,CASE 
            WHEN Medal = 'Silver'
                THEN '1'
            ELSE 0
            END AS Silver
        ,CASE 
            WHEN Medal = 'Bronze'
                THEN '1'
            ELSE 0
            END AS Bronze
    FROM olympians
    )
    ,data
AS (
    SELECT Sex
        ,(sum(Gold) + Sum(Silver) + sum(Bronze)) AS Total
        ,sum(Gold) AS Gold
        ,sum(Silver) AS Silver
        ,sum(Bronze) AS Bronze
    FROM if_else_cases
    GROUP BY Sex
    )
SELECT *
FROM data

"""
run_query(medals_by_sex)

,Sex,Total,Gold,Silver,Bronze
0,Female,11253,3747,3735,3771
1,Male,28530,9625,9381,9524


##### Which countries have never won a Gold medal but have won Bronze or Silver Medals?

In [32]:
goldless_countries = """

SELECT *
FROM (
    SELECT c.region
        ,COUNT(CASE 
                WHEN Medal = 'Gold'
                    THEN 1
                END) AS Gold
        ,COUNT(CASE 
                WHEN Medal = 'Silver'
                    THEN 1
                END) AS Silver
        ,COUNT(CASE 
                WHEN Medal = 'Bronze'
                    THEN 1
                END) AS Bronze
    FROM olympians AS athletes
    INNER JOIN countries AS c ON athletes.NOC = c.NOC
    GROUP BY region
    ORDER BY region
    ) AS tb1
WHERE Gold = 0
    AND (
        Silver > 0
        OR Bronze > 0
        )
    ORDER BY (Silver + Bronze) DESC;
    
"""
run_query(goldless_countries)

,region,Gold,Silver,Bronze
0,Ghana,0,1,22
1,Iceland,0,15,2
2,Paraguay,0,17,0
3,Malaysia,0,11,5
4,Montenegro,0,14,0
5,Philippines,0,3,7
6,Moldova,0,3,5
7,Saudi Arabia,0,1,5
8,Qatar,0,1,4
9,Lebanon,0,2,2


##### In what events has the USA won the most medals?

In [33]:
usa_most_medals = """

SELECT Sport
    ,COUNT(Medal) AS Medals
FROM olympians
WHERE noc = 'USA'
GROUP BY Event
ORDER BY Medals DESC LIMIT 10;

"""
run_query(usa_most_medals)

,Sport,Medals
0,Basketball,222
1,Ice Hockey,175
2,Swimming,147
3,Rowing,144
4,Swimming,126
5,Basketball,119
6,Swimming,108
7,Swimming,108
8,Ice Hockey,101
9,Athletics,94
